<a href="https://colab.research.google.com/github/gentleman101/Deep_Learning/blob/main/carnivores_image_classification_using_pre_trained_model_(Keras).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Necessary Packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

## Importing Pre-Trained model

In [4]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model,load_model
# importing inception net from Keras with imagenet weights
conv_base =  InceptionV3(weights='imagenet',include_top=False,input_shape=(300, 300, 3))

87916544/87910968 [==============================] - 4s 0us/step


In [5]:
output = conv_base.layers[-1].output
output = keras.layers.Flatten()(output)
model_tl = Model(conv_base.input, output)


In [6]:
model_tl.trainable = False
for layer in model_tl.layers:
  layer.trainable = False
layers = [(layer, layer.name, layer.trainable) for layer in  model_tl.layers]
model_layers=pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])
print(model_layers) 


                                            Layer Type  ... Layer Trainable
0    <tensorflow.python.keras.engine.input_layer.In...  ...           False
1    <tensorflow.python.keras.layers.convolutional....  ...           False
2    <tensorflow.python.keras.layers.normalization_...  ...           False
3    <tensorflow.python.keras.layers.core.Activatio...  ...           False
4    <tensorflow.python.keras.layers.convolutional....  ...           False
..                                                 ...  ...             ...
307  <tensorflow.python.keras.layers.merge.Concaten...  ...           False
308  <tensorflow.python.keras.layers.merge.Concaten...  ...           False
309  <tensorflow.python.keras.layers.core.Activatio...  ...           False
310  <tensorflow.python.keras.layers.merge.Concaten...  ...           False
311  <tensorflow.python.keras.layers.core.Flatten o...  ...           False

[312 rows x 3 columns]


## Data Pre-Processing

In [7]:
# Data Augmentation
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [8]:
test_size=400
batch_size=32
epochs=25

In [10]:
test_path='/content/drive/MyDrive/DATA/Carnivores_Data/test'
train_path='/content/drive/MyDrive/DATA/Carnivores_Data/train'

In [17]:
# Initiationg ImageDataGenerator on Keras
target_size=(300,300) #resize all images to 300x300
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3,  
rotation_range=50,
width_shift_range=0.2, 
 height_shift_range=0.2, 
shear_range=0.2,
horizontal_flip=True,
brightness_range = [0.8, 1.2],
fill_mode='nearest',        
 validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)


In [18]:
# The list of classes will be automatically inferred from the subdirectory names/structure under train_dir
train_generator = train_datagen.flow_from_directory(
train_path,
target_size=target_size,#  
batch_size=batch_size,
class_mode='categorical',
subset='training')
validation_generator = train_datagen.flow_from_directory(
train_path,
target_size=target_size,
batch_size=batch_size,
class_mode='categorical',
subset='validation')

Found 2899 images belonging to 1 classes.
Found 724 images belonging to 1 classes.


## Building Model Architecture

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras import optimizers
# building a linear stack of layers with the sequential model
model =Sequential()
model.add(model_tl)
# hidden layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
# output layer
model.add(Dense(4, activation='softmax'))
# compiling the sequential model
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 131072)            21802784  
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16777344  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 516       
Total params: 38,580,644
Trainable params: 16,777,860
Non-trainable params: 21,802,784
_________________________________________________________________
None


Creating Checkpoints so to recover if sessiosn is disconected

In [20]:
from keras.callbacks import *
filepath="/content/drive/My Drive/MyCNN/epochs:{epoch:03d}-val_acc: {val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=False,save_freq='epoch',mode='max')
callbacks_list = [checkpoint]

## Training the model

In [ ]:
history = model.fit(train_generator,steps_per_epoch=train_generator.samples//batch_size,validation_data=validation_generator,
validation_steps=validation_generator.samples//batch_size,
epochs=epochs,verbose=1,shuffle=True,callbacks=callbacks_list)

Epoch 1/25
 9/90 [==>...........................] - ETA: 37:51 - loss: 37.1194 - acc: 0.3454